In [1]:
import numpy as np
import os, glob
import matplotlib.pyplot as plt
plt.rc("font",size=14)

from netCDF4 import Dataset
import xarray, wrf

from pandas import to_datetime
from scipy.signal import welch
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
from matplotlib.ticker import FormatStrFormatter

In [2]:
plt.rc("font",size=14)

In [3]:
from scipy import fftpack

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
height = 100.0

In [6]:
dx = {"MYNN":333.0,
       "SH":333.0,
       "VLES":333.0,
       "YSU":333.0,
       "LES_25":25.0,
       "LES_100":100.0}
segment_length = 30000.0

In [7]:
colors = {"LES_25":"k","LES_100":"k","MYNN":"#D4AC0D","SH":"#C0392B","YSU":"#2980B9","VLES":"#117A65"}

lws = {"LES_25":1,"LES_100":2,"MYNN":4,"SH":2,"YSU":1,"VLES":2}

lss = {"LES_25":"-","LES_100":":","MYNN":"-","SH":"-","YSU":"--","VLES":"-."}

In [8]:
day      = 21

In [9]:
directories = {"MYNN":"/glade/scratch/doubrawa/MYNN/03{0}15/".format(day),
               "SH":"/glade/scratch/doubrawa/SH/03{0}15/".format(day),
               "VLES":"/glade/scratch/doubrawa/VLES/03{0}15/".format(day),
               "YSU":"/glade/scratch/doubrawa/YSU/03{0}15/".format(day),
               "LES_25":"/glade/scratch/doubrawa/postProcessing/",
               "LES_100":"/glade/scratch/doubrawa/postProcessing/"}

In [10]:
figPath = "/glade/u/home/doubrawa/figs/"

### define a time

In [11]:
year     = 2015
month    = 3
day      = 21
for hour in range(14,24):
    for minute in [0,30]:
        
        if ( (day==28) & (hour==19) & (minute==30) ):
            break
        
        datetime = to_datetime("{0}-{1}-{2} {3}:{4}".format(year,month,day,hour,minute),format="%Y-%m-%d %H:%M")
        #
        #
        #
        # read in the simulations I ran
        #
        #
        #
        w     = {}
        xlat  = {}
        xlong = {}

        for key,directory in directories.items():

            if "LES_" not in key:

                print ("**********************************")
                print (key)

                wrfouts = sorted(glob.glob(os.path.join(directory,'wrfout_d03*')))
                ifile   = 0
                for wrfout in wrfouts:
                    # from the file name, figure out what time these data are for
                    time = to_datetime(wrfout.split("_d03_")[-1],format="%Y-%m-%d_%H:%M:%S")

                    # only proceed if after 1400 UTC (1200-1400 is spin up)
                    if ((time-datetime).seconds < 5 * 60.0):
                        print (os.path.split(wrfout)[-1])

                        wrfnc = Dataset(wrfout)

                        #
                        # these are needed for vertical interpolation
                        #
                        if ifile==0:
                            hgt  = wrf.getvar(wrfnc, "ter", units="m")
                            z    = wrf.getvar(wrfnc, "z", units="m")
                            zref = z-hgt

                        w_3d = wrf.getvar(wrfnc, "wa", units="m s-1", timeidx=wrf.ALL_TIMES)
                        w_2d = wrf.interplevel(w_3d, zref, height, meta=True)    

                        if ifile==0:
                            w[key] = w_2d
                        else:
                            w[key] = xarray.concat([w[key], w_2d], dim='Time')

                        if ifile==0:
                            xlat[key]  = wrf.getvar(wrfnc, "lat")
                            xlong[key] = wrf.getvar(wrfnc, "lon")

                        ifile += 1
        #
        #
        #
        # read in the simulations Domingo ran
        #
        #
        #
        for key,directory in directories.items():

            if "LES_" in key:

                print ("**********************************")
                print (key)

                wrfouts = sorted(glob.glob(os.path.join(directory,'WRF_{0}m_2015-03-{1}*.nc'.format(key,day))))
                ifile   = 0
                for wrfout in wrfouts:

                    wrfnc = xarray.open_dataset(wrfout)
                    time  = to_datetime(wrfnc.time[0].data)

                    if ((time-datetime).seconds < 5 * 60.0):
                        print (os.path.split(wrfout)[-1])                

                        if ifile==0:
                            w[key] = wrfnc["w"]
                        else:
                            w[key] = xarray.concat([w[key], wrfnc["w"]], dim='time')

                        if ifile==0:
                            xlat[key]  = wrfnc["xlat"]
                            xlong[key] = wrfnc["xlong"]

                        ifile += 1        

        for key in ["MYNN","YSU","SH","VLES"]:
            w[key]["time"] = w[key]["Time"]
            w[key] = w[key].expand_dims('time')
        #
        #
        #
        # plot each simulation method, just to make sure it's all there
        #
        #
        #

        levels = np.arange(-1.0,1.1,0.1)

        fig = plt.figure(figsize=(15,10))

        ax  = {}
        for iax,key in enumerate(["LES_25","LES_100","VLES","MYNN","YSU","SH"]):

            if iax==0:
                ax[iax] = fig.add_subplot(2,3,iax+1)
            else:
                ax[iax] = fig.add_subplot(2,3,iax+1,sharex=ax[0],sharey=ax[0])
            ax[iax].set_title(key)

            try:
                contour = w[key].sel(time=datetime)
            except:
                datetimes_available = w[key]["time"]
                dt_offsets = np.asarray([ (to_datetime(a)-datetime).seconds for a in datetimes_available.data])
                if (np.min(dt_offsets) < 60.0):
                    idx   = np.argmin(dt_offsets)
                    avail = to_datetime(datetimes_available.data[idx])
                    contour = w[key].sel(time=avail)
                    print ("Couldn't find {0:%Y-%m-%d %H:%M:%S}, settling for {1:%Y-%m-%d %H:%M:%S}".format(datetime,avail))

            lat = xlat[key]
            lon = xlong[key]

            p = ax[iax].contourf(lon,lat,contour,levels=levels,cmap="RdBu")
            
            if iax not in [0,3]:
                plt.setp(ax[iax].get_yticklabels(), visible=False)
            if iax not in [3,4,5]:
                plt.setp(ax[iax].get_xticklabels(), visible=False)                   
            
        fig.subplots_adjust(right=0.8)
        cbar_ax = fig.add_axes([0.82, 0.255, 0.02, 0.49])
        clb = fig.colorbar(p, cax=cbar_ax)
        clb.set_label(r'Vertical Velocity $w$ [m/s]', labelpad=15, y=0.45)  
        
        #fig.savefig(os.path.join(figPath,"contours_w_{0:%Y-%m-%d-%H-%M}.png".format(datetime)),bbox_to_inches="tight")        

        ### equivalent x/y coordinates across domains

        x = {}
        y = {}

        ref_wrfnc = Dataset("/glade/scratch/domingom/Cheyenne/XPIA_mesoLES/SIMULS/WRF_mesoLES_4dom_RAP_2015_03_12_mesoLES/HOUR_14_1/wrfout_d04_2015-03-13_14:00:10_0000")

        for key in directories.keys():

            x[key], y[key] = wrf.ll_to_xy(ref_wrfnc, xlat[key], xlong[key])
            x[key]   = np.reshape(x[key].data,w[key].shape[1:])
            y[key]   = np.reshape(y[key].data,w[key].shape[1:])

        #
        #
        #
        # clip all of them to the same area
        #
        #
        #            
        xmin, xmax = np.min(x["LES_25"]), np.max(x["LES_25"])
        ymin, ymax = np.min(y["LES_25"]), np.max(y["LES_25"])

        x_clipped   = {}
        y_clipped   = {}
        w_clipped   = {}
        lat_clipped = {}
        lon_clipped = {}

        for key in ["LES_100","MYNN","SH","YSU","VLES"]:

            print (key)

            condition_x = ( (x[key] >= xmin) & (x[key]<=xmax) )
            condition_y = ( (y[key] >= ymin) & (y[key]<=ymax) )
            condition   = condition_x & condition_y

            idx_sn, idx_we = np.where(condition)
            idx_sn = np.unique(idx_sn)
            idx_we = np.unique(idx_we)

            w_clipped[key]   = (w[key].isel(west_east=idx_we,south_north=idx_sn)).copy()
            x_clipped[key]   = (x[key][idx_sn,idx_we]).copy()
            y_clipped[key]   = (y[key][idx_sn,idx_we]).copy()
            lat_clipped[key] = (xlat[key][idx_sn,idx_we]).copy()
            lon_clipped[key] = (xlong[key][idx_sn,idx_we]).copy()

        x_clipped["LES_25"] = x["LES_25"]
        y_clipped["LES_25"] = y["LES_25"]
        w_clipped["LES_25"] = w["LES_25"]
        
        lat_clipped["LES_25"] = xlat["LES_25"]
        lon_clipped["LES_25"] = xlong["LES_25"]        

        levels = np.arange(-1.0,1.1,0.1)

        fig   = plt.figure(figsize=(15,10))

        axbig = fig.add_subplot(111)
        axbig.spines['top'].set_color('none')
        axbig.spines['bottom'].set_color('none')
        axbig.spines['left'].set_color('none')
        axbig.spines['right'].set_color('none')
        axbig.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
        axbig.set_xlabel("west_east [-]",labelpad=10)
        axbig.set_ylabel("south_north [-]",labelpad=15)
        axbig.set_title("{0:%Y-%m-%d %H:%M}".format(datetime),y=1.08)

        ax  = {}
        for iax,key in enumerate(["LES_25","LES_100","VLES","MYNN","YSU","SH"]):

            if iax==0:
                ax[iax] = fig.add_subplot(2,3,iax+1,aspect="equal")
            else:
                ax[iax] = fig.add_subplot(2,3,iax+1,sharex=ax[0],sharey=ax[0],aspect="equal")
            ax[iax].set_title(key)

            try:
                contour = w_clipped[key].sel(time=datetime)
            except:
                print("skipping...")
                datetimes_available = w_clipped[key]["time"]
                dt_offsets = np.asarray([ (to_datetime(a)-datetime).seconds for a in datetimes_available.data])
                if (np.min(dt_offsets) < 2*60.0):
                    idx   = np.argmin(dt_offsets)
                    avail = to_datetime(datetimes_available.data[idx])
                    contour = w_clipped[key].sel(time=avail)
                print ("Couldn't find {0:%Y-%m-%d %H:%M:%S}, settling for {1:%Y-%m-%d %H:%M:%S}".format(datetime,avail))

            xx = x_clipped[key]
            yy = y_clipped[key]

            p = ax[iax].contourf(xx,yy,contour,levels=levels,cmap="RdBu")
   
            ax[iax].xaxis.set_tick_params(direction='in')
            ax[iax].yaxis.set_tick_params(direction='in')   

            if iax not in [0,3]:
                plt.setp(ax[iax].get_yticklabels(), visible=False)
            if iax not in [3,4,5]:
                plt.setp(ax[iax].get_xticklabels(), visible=False)        

        fig.subplots_adjust(right=0.8)
        cbar_ax = fig.add_axes([0.82, 0.255, 0.02, 0.49])
        clb = fig.colorbar(p, cax=cbar_ax)
        clb.set_label(r'Vertical Velocity $w$ [m/s]', labelpad=15, y=0.45)   

        #fig.savefig(os.path.join(figPath,"contours_w_clipped_{0:%Y-%m-%d-%H-%M}.png".format(datetime)),bbox_to_inches="tight")    
        break

#         #
#         #
#         #
#         # compare their spectra -- 1-D
#         #
#         #
#         #
        
#         #
#         #
#         #
#         # along south_north direction
#         #
#         #
#         #
    
#         fig = plt.figure(figsize=(15,5))

#         axbig = fig.add_subplot(111)
#         axbig.spines['top'].set_color('none')
#         axbig.spines['bottom'].set_color('none')
#         axbig.spines['left'].set_color('none')
#         axbig.spines['right'].set_color('none')
#         axbig.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
#         axbig.set_xlabel("$\kappa$ [1/m] ",labelpad=8)
#         axbig.set_ylabel("PSD of $w$ [(m/s)**2 / (1/m)] ",labelpad=15)
#         axbig.set_title("{0:%Y-%m-%d %H:%M}".format(datetime),y=1.05)

#         ax  = fig.add_subplot(1,2,1)

#         for key in ["LES_25","LES_100","MYNN","SH","YSU","VLES"]:

#             try:
#                 contour = w[key].sel(time=datetime)
#             except:
#                 print("skipping...")
#                 datetimes_available = w[key]["time"]
#                 dt_offsets = np.asarray([ (to_datetime(a)-datetime).seconds for a in datetimes_available.data])
#                 if (np.min(dt_offsets) < 2*60.0):
#                     idx   = np.argmin(dt_offsets)
#                     avail = to_datetime(datetimes_available.data[idx])
#                     contour = w[key].sel(time=avail)
#                 print ("Couldn't find {0:%Y-%m-%d %H:%M:%S}, settling for {1:%Y-%m-%d %H:%M:%S}".format(datetime,avail))

#             xx = contour.interpolate_na(dim='south_north').interpolate_na(dim='west_east')

#             nseg    = segment_length/dx[key]         

#             n_west_east = len(xx.west_east)
#             for i in range(n_west_east):

#                 x_south_north = xx.data[:,i]

#                 if i==0:
#                     ff, pp = np.abs(welch(x_south_north,fs=1/dx[key],nperseg=nseg))
#                 else:
#                     junk, now = np.abs(welch(x_south_north,fs=1/dx[key],nperseg=nseg))
#                     pp = pp + now

#             pp = pp/float(n_west_east)    

#             label = "$\Delta={0:.0f}$ m ({1})".format(dx[key],key.split("_")[0])
#             ax.loglog(ff[1:],pp[1:],label=label,color=colors[key],lw=lws[key],ls=lss[key],alpha=1)   

#         ax.axvline(x=1/666.0,color='gray',linestyle=":",lw=1)
#         ax.axvline(x=1/200.0,color='gray',linestyle=":",lw=1)
#         ax.axvline(x=1/50.0,color='gray',linestyle=":",lw=1)
#         ax.axvline(x=1/segment_length, color='gray',linestyle=":",lw=1)        

#         ax.set_xlim([1e-5,5e-2])
#         ax.set_ylim([1e-4,1e4])

#         ref_f     = np.linspace(4e-3,2e-2,20)
#         ref_slope = ref_f**(-5/3) * 5e-1
#         ax.loglog(ref_f,ref_slope,'--k',lw=2)
#         ax.text(1e-2,1.2e3,'$\propto \kappa^{{-5/3}}$',horizontalalignment='left') 

#         kappas = [50.,200.,666.,5000.0,30000.0]
#         ax.set_xticks([1/q for q in kappas])
#         ax.set_xticklabels([r"{0:.0f}$^{{-1}}$".format(q) for q in kappas]) 

#         plt.legend(loc=3,fontsize=10)

#         #
#         # CLIPPED
#         #

#         ax2 = fig.add_subplot(1,2,2,sharex=ax,sharey=ax)

#         for key in ["LES_25","LES_100","MYNN","SH","YSU","VLES"]:

#             try:
#                 contour = w_clipped[key].sel(time=datetime)
#             except:
#                 print("skipping...")
#                 datetimes_available = w_clipped[key]["time"]
#                 dt_offsets = np.asarray([ (to_datetime(a)-datetime).seconds for a in datetimes_available.data])
#                 if (np.min(dt_offsets) < 2*60.0):
#                     idx   = np.argmin(dt_offsets)
#                     avail = to_datetime(datetimes_available.data[idx])
#                     contour = w_clipped[key].sel(time=avail)
#                 print ("Couldn't find {0:%Y-%m-%d %H:%M:%S}, settling for {1:%Y-%m-%d %H:%M:%S}".format(datetime,avail))
#             xx = contour.interpolate_na(dim='south_north').interpolate_na(dim='west_east')

#             nseg    = segment_length/dx[key]                     
#             n_west_east = len(xx.west_east)
#             for i in range(n_west_east):

#                 x_south_north = xx.data[:,i]

#                 if i==0:
#                     ff, pp = np.abs(welch(x_south_north,fs=1/dx[key],nperseg=nseg))
#                 else:
#                     junk, now = np.abs(welch(x_south_north,fs=1/dx[key],nperseg=nseg))
#                     pp = pp + now

#             pp = pp/float(n_west_east)    

#             ax2.loglog(ff[1:],pp[1:],label=label,color=colors[key],lw=lws[key],ls=lss[key],alpha=1)   

#         ax2.axvline(x=1/666.0,color='gray',linestyle=":",lw=1)
#         ax2.axvline(x=1/200.0,color='gray',linestyle=":",lw=1)
#         ax2.axvline(x=1/50.0,color='gray',linestyle=":",lw=1)
#         ax2.axvline(x=1/segment_length, color='gray',linestyle=":",lw=1)   
#         plt.setp(ax2.get_yticklabels(), visible=False)

#         ax2.loglog(ref_f,ref_slope,'--k',lw=2)
#         ax2.text(1e-2,1.2e3,'$\propto \kappa^{{-5/3}}$',horizontalalignment='left')         

#         kappas = [50.,200.,666.,5000.0,30000.0]
#         ax2.set_xticks([1/q for q in kappas])
#         ax2.set_xticklabels([r"{0:.0f}$^{{-1}}$".format(q) for q in kappas])         
        
#         fig.savefig(os.path.join(figPath,"PSD_1d_w_along_south_north_{0:%Y-%m-%d-%H-%M}.png".format(datetime)),bbox_to_inches="tight")

#         #
#         # along west_east direction
#         #

#         fig = plt.figure(figsize=(15,5))

#         axbig = fig.add_subplot(111)
#         axbig.spines['top'].set_color('none')
#         axbig.spines['bottom'].set_color('none')
#         axbig.spines['left'].set_color('none')
#         axbig.spines['right'].set_color('none')
#         axbig.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
#         axbig.set_xlabel("$\kappa$ [1/m] ",labelpad=8)
#         axbig.set_ylabel("PSD of $w$ [(m/s)**2 / (1/m)] ",labelpad=15)
#         axbig.set_title("{0:%Y-%m-%d %H:%M}".format(datetime),y=1.05)

#         ax  = fig.add_subplot(1,2,1)

#         for key in ["LES_25","LES_100","MYNN","SH","YSU","VLES"]:

#             try:
#                 contour = w[key].sel(time=datetime)
#             except:
#                 print("skipping...")
#                 datetimes_available = w[key]["time"]
#                 dt_offsets = np.asarray([ (to_datetime(a)-datetime).seconds for a in datetimes_available.data])
#                 if (np.min(dt_offsets) < 2*60.0):
#                     idx   = np.argmin(dt_offsets)
#                     avail = to_datetime(datetimes_available.data[idx])
#                     contour = w[key].sel(time=avail)
#                 print ("Couldn't find {0:%Y-%m-%d %H:%M:%S}, settling for {1:%Y-%m-%d %H:%M:%S}".format(datetime,avail))

#             xx = contour.interpolate_na(dim='south_north').interpolate_na(dim='west_east')
#             nseg    = segment_length/dx[key]         

#             n_south_north = len(xx.south_north)
#             for i in range(n_south_north):

#                 x_west_east = xx.data[i,:]

#                 if i==0:
#                     ff, pp = np.abs(welch(x_west_east,fs=1/dx[key],nperseg=nseg))
#                 else:
#                     junk, now = np.abs(welch(x_west_east,fs=1/dx[key],nperseg=nseg))
#                     pp = pp + now

#             pp = pp/float(n_south_north)    

#             label = "$\Delta={0:.0f}$ m ({1})".format(dx[key],key.split("_")[0])
#             ax.loglog(ff[1:],pp[1:],label=label,color=colors[key],lw=lws[key],ls=lss[key],alpha=1)   

#         ax.axvline(x=1/666.0,color='gray',linestyle=":",lw=1)
#         ax.axvline(x=1/200.0,color='gray',linestyle=":",lw=1)
#         ax.axvline(x=1/50.0,color='gray',linestyle=":",lw=1)
#         ax.axvline(x=1/segment_length, color='gray',linestyle=":",lw=1)   

#         ax.set_xlim([1e-5,5e-2])
#         ax.set_ylim([1e-4,1e4])            

#         ref_f     = np.linspace(4e-3,2e-2,20)
#         ref_slope = ref_f**(-5/3) * 5e-1
#         ax.loglog(ref_f,ref_slope,'--k',lw=2)
#         ax.text(1e-2,1.2e3,'$\propto \kappa^{{-5/3}}$',horizontalalignment='left') 

#         kappas = [50.,200.,666.,5000.0,30000.0]
#         ax.set_xticks([1/q for q in kappas])
#         ax.set_xticklabels([r"{0:.0f}$^{{-1}}$".format(q) for q in kappas])

#         plt.legend(loc=3,fontsize=10)

#         #
#         # CLIPPED
#         #

#         ax2 = fig.add_subplot(1,2,2,sharex=ax,sharey=ax)

#         for key in ["LES_25","LES_100","MYNN","SH","YSU","VLES"]:

#             try:
#                 contour = w_clipped[key].sel(time=datetime)
#             except:
#                 print("skipping...")
#                 datetimes_available = w_clipped[key]["time"]
#                 dt_offsets = np.asarray([ (to_datetime(a)-datetime).seconds for a in datetimes_available.data])
#                 if (np.min(dt_offsets) < 2*60.0):
#                     idx   = np.argmin(dt_offsets)
#                     avail = to_datetime(datetimes_available.data[idx])
#                     contour = w_clipped[key].sel(time=avail)
#                 print ("Couldn't find {0:%Y-%m-%d %H:%M:%S}, settling for {1:%Y-%m-%d %H:%M:%S}".format(datetime,avail))
#             xx = contour.interpolate_na(dim='south_north').interpolate_na(dim='west_east')

#             nseg    = segment_length/dx[key]         
#             n_south_north = len(xx.south_north)
#             for i in range(n_south_north):

#                 x_west_east = xx.data[i,:]

#                 if i==0:
#                     ff, pp = np.abs(welch(x_west_east,fs=1/dx[key],nperseg=nseg))
#                 else:
#                     junk, now = np.abs(welch(x_west_east,fs=1/dx[key],nperseg=nseg))
#                     pp = pp + now

#             pp = pp/float(n_south_north)    
            
#             ax2.loglog(ff[1:],pp[1:],label=label,color=colors[key],lw=lws[key],ls=lss[key],alpha=1)   

#         ax2.axvline(x=1/666.0,color='gray',linestyle=":",lw=1)
#         ax2.axvline(x=1/200.0,color='gray',linestyle=":",lw=1)
#         ax2.axvline(x=1/50.0,color='gray',linestyle=":",lw=1)
#         ax2.axvline(x=1/segment_length, color='gray',linestyle=":",lw=1)   
#         plt.setp(ax2.get_yticklabels(), visible=False)
        
#         ax2.loglog(ref_f,ref_slope,'--k',lw=2)
#         ax2.text(1e-2,1.2e3,'$\propto \kappa^{{-5/3}}$',horizontalalignment='left')         
        
#         kappas = [50.,200.,666.,5000.0,30000.0]
#         ax2.set_xticks([1/q for q in kappas])
#         ax2.set_xticklabels([r"{0:.0f}$^{{-1}}$".format(q) for q in kappas])        

#         fig.savefig(os.path.join(figPath,"PSD_1d_w_along_west_east_{0:%Y-%m-%d-%H-%M}.png".format(datetime)),bbox_to_inches="tight")

**********************************
MYNN
**********************************
SH
**********************************
VLES
**********************************
YSU
**********************************
LES_25
**********************************
LES_100


KeyError: 'MYNN'